이것은 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 책의 노트북이다. 가독성을 위해 실행 가능한 코드 블록과 섹션 제목만 포함하고 책의 다른 모든 텍스트 단락, 그림 및 의사 코드는 생략한다.

**무슨 일이 일어나고 있는지 확인하고 싶다면 책과 함께 노트북을 사용하는 것을 권장한다.**

이 노트북은 TensorFlow 2.6 기반이다. (옮긴이: 버전 2.10.0, 애플 실리콘 M1 으로 테스트)

# 텍스트를 위한 딥 러닝

## 자연어 처리: 조감도

## 테스트 데이터 준비

### 텍스트 표준화

### 텍스트 분할 (토큰화, tokenization)

### 어휘 색인

### `TextVectorization` 층 사용

In [1]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [2]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [3]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [4]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    output_mode="int",
)

Metal device set to: Apple M1 Pro


2022-11-16 22:52:07.210570: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-16 22:52:07.210684: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

In [6]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

2022-11-16 22:52:09.929253: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-16 22:52:09.982086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


**어휘 표시**

In [7]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [8]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [9]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


## 단어 그룹 표현의 두 가지 접근법: 집합(Set)과 시퀀스(Sequence)

### IMDB 영화 리뷰 데이터 준비

In [10]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  1374k      0  0:00:59  0:00:59 --:--:-- 1925k1:00  0:00:36  0:00:24 1345k1  0:00:11 1186k


In [11]:
!rm -r aclImdb/train/unsup

In [12]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [13]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [14]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**첫 번째 배치의 형태(shape)와 데이터타입(dtype) 표시**

In [15]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Playing a character from a literary classic can be a bit of a poisoned chalice for an actor, paying for the pleasure of a meaty character by competing with the fantasies of generations of readers \xc2\x96 not to mention the numerous other actors who\'ve besieged the castle before. Fortunately for the fantasists, this version \xc2\x96 with the nicely cast Zelah Clarke and Timothy Dalton \xc2\x96 stands head and shoulders above versions that have come after it. It\'s the right length to do the story full justice, and makes considerable use of Bronte\'s cracking dialogue; none of that modern meddling away, cutting text and adding new and inferior scenes.<br /><br />The magic of the original story lies in the tensions created between the central characters, and the lives circumstances create for them to lead. Jane \xc2\x96 "poor, plain and little" \xc2\x96 grows u

### 단어를 집합으로 처리: Bag-of-Words(BoW) 접근법

#### 이진 인코딩을 가진 단일 단어 (unigrams)

**`TextVectorization` 층으로 데이터셋 전처리**

In [16]:
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

2022-11-16 23:06:18.457132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


**이전 유니그램(unigram) 데이터셋의 출력 조사**

In [17]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([0. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


**모델 구축 유틸리티**

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

**이진 유니그램 모델 훈련과 테스트**

In [19]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-11-16 23:08:20.989380: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


621/625 [============================>.] - ETA: 0s - loss: 0.3891 - accuracy: 0.8390

2022-11-16 23:08:28.087811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 8s 10ms/step - loss: 0.3885 - accuracy: 0.8391 - val_loss: 0.2697 - val_accuracy: 0.8950
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.1949 - accuracy: 0.9307 - val_loss: 0.2618 - val_accuracy: 0.8982
Epoch 3/10
625/625 [==============================] - 6s 9ms/step - loss: 0.1253 - accuracy: 0.9605 - val_loss: 0.2797 - val_accuracy: 0.8988
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0872 - accuracy: 0.9749 - val_loss: 0.3106 - val_accuracy: 0.8962
Epoch 5/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0629 - accuracy: 0.9823 - val_loss: 0.3460 - val_accuracy: 0.8938
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0464 - accuracy: 0.9872 - val_loss: 0.3847 - val_accuracy: 0.8942
Epoch 7/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0349 - accuracy: 0.9904 - val_loss: 0.4284 - val_accuracy: 0.8956
Epoch 8/10
625/6

2022-11-16 23:09:20.234762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 5s 6ms/step - loss: 0.2905 - accuracy: 0.8874
Test acc: 0.887


#### 이진 인코딩을 가진 바이그램(bigram)

**바이그램을 반환하도록 `TextVectorization` 층 구성**

In [20]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

**이진 바이그램 모델 훈련과 테스트**

In [21]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

2022-11-16 23:12:58.146297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  1/625 [..............................] - ETA: 5:14 - loss: 0.7040 - accuracy: 0.5625

2022-11-16 23:13:05.951880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


619/625 [============================>.] - ETA: 0s - loss: 0.3696 - accuracy: 0.8462

2022-11-16 23:13:11.236351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 7s 10ms/step - loss: 0.3700 - accuracy: 0.8462 - val_loss: 0.2529 - val_accuracy: 0.9046
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.1589 - accuracy: 0.9480 - val_loss: 0.2575 - val_accuracy: 0.9082
Epoch 3/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0940 - accuracy: 0.9720 - val_loss: 0.2954 - val_accuracy: 0.9094
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0634 - accuracy: 0.9832 - val_loss: 0.3555 - val_accuracy: 0.9104
Epoch 5/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0474 - accuracy: 0.9881 - val_loss: 0.4181 - val_accuracy: 0.9094
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0384 - accuracy: 0.9904 - val_loss: 0.4841 - val_accuracy: 0.9036
Epoch 7/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0319 - accuracy: 0.9922 - val_loss: 0.5546 - val_accuracy: 0.9034
Epoch 8/10
625/6

2022-11-16 23:14:03.161531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 5s 7ms/step - loss: 0.2749 - accuracy: 0.8946
Test acc: 0.895


#### TF-IDF 인코딩을 가진 바이그램

**토큰 개수를 반환하도록 `TextVectorization` 층 구성**

In [22]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

**TF-IDF-가중치가 적용되어 반환하도록 TextVectorization 층 구성**

In [23]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

**TF-IDF 바이그램 모델을 훈련하고 테스트**

In [24]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

2022-11-16 23:20:54.421026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  1/625 [..............................] - ETA: 3:43 - loss: 8.4963 - accuracy: 0.5312

2022-11-16 23:21:13.396550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


621/625 [============================>.] - ETA: 0s - loss: 0.5058 - accuracy: 0.7970

2022-11-16 23:21:19.358196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 7s 11ms/step - loss: 0.5062 - accuracy: 0.7968 - val_loss: 0.2946 - val_accuracy: 0.8836
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2470 - accuracy: 0.9076 - val_loss: 0.3608 - val_accuracy: 0.8744
Epoch 3/10
625/625 [==============================] - 6s 9ms/step - loss: 0.1657 - accuracy: 0.9390 - val_loss: 0.3315 - val_accuracy: 0.8972
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.1254 - accuracy: 0.9528 - val_loss: 0.4023 - val_accuracy: 0.8870
Epoch 5/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0987 - accuracy: 0.9611 - val_loss: 0.4113 - val_accuracy: 0.8964
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0781 - accuracy: 0.9696 - val_loss: 0.4900 - val_accuracy: 0.8950
Epoch 7/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0665 - accuracy: 0.9742 - val_loss: 0.5477 - val_accuracy: 0.8884
Epoch 8/10
625/6

2022-11-16 23:22:11.814520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 5s 7ms/step - loss: 0.3162 - accuracy: 0.8687
Test acc: 0.869


In [25]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [26]:
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

81.85 percent positive
